# Outlier detection using UMAP

In [1]:
import numpy as np
import sklearn.datasets
import sklearn.neighbors
import umap
import umap.plot
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data, labels = sklearn.datasets.fetch_openml('mnist_784', version=1, return_X_y=True)

MemoryError: 

In [ ]:
%%time
outlier_scores = sklearn.neighbors.LocalOutlierFactor(contamination=0.001428).fit_predict(data)

In [ ]:
outlying_digits = data[outlier_scores == -1]
outlying_digits.shape

In [ ]:
fig, axes = plt.subplots(7, 10, figsize=(10,10))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(outlying_digits[i].reshape((28,28)))
    plt.setp(ax, xticks=[], yticks=[])
plt.tight_layout()

In [ ]:
mapper = umap.UMAP().fit(data)

In [ ]:
umap.plot.points(mapper, labels=labels)

In [ ]:
outlier_scores = sklearn.neighbors.LocalOutlierFactor(contamination=0.001428).fit_predict(mapper.embedding_)

In [ ]:
outlying_digits = data[outlier_scores == -1]
outlying_digits.shape

In [ ]:
fig, axes = plt.subplots(7, 10, figsize=(10,10))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(outlying_digits[i].reshape((28,28)))
    plt.setp(ax, xticks=[], yticks=[])
plt.tight_layout()

In [ ]:
mapper = umap.UMAP(set_op_mix_ratio=0.25).fit(data)

In [ ]:
umap.plot.points(mapper, labels=labels)

In [ ]:
outlier_scores = sklearn.neighbors.LocalOutlierFactor(contamination=0.001428).fit_predict(mapper.embedding_)

In [ ]:
outlying_digits = data[outlier_scores == -1]
outlying_digits.shape

In [ ]:
fig, axes = plt.subplots(10, 10, figsize=(10,10))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(outlying_digits[i].reshape((28,28)))
    plt.setp(ax, xticks=[], yticks=[])
plt.tight_layout()